In [1]:
import pandas as pd
import json
import datetime 
import time
import numpy as np

## Users

In [2]:
df_users = pd.read_json("C:/Users/keert/Downloads/json_data/receipts.json/users.json/users.json", lines=True)

In [3]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [4]:
df_users['createdDate'] = df_users['createdDate'].str.get('$date')
df_users['_id'] = df_users['_id'].str.get('$oid')
df_users['lastLogin'] = df_users['lastLogin'].str.get('$date')

In [5]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [6]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           495 non-null    object 
 1   active        495 non-null    bool   
 2   createdDate   495 non-null    int64  
 3   lastLogin     433 non-null    float64
 4   role          495 non-null    object 
 5   signUpSource  447 non-null    object 
 6   state         439 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [7]:
#createdDate, lastLogin is in UNIX / Epoch time format
#lastLogin is float datatype. We will have to convert it to int in order to be able to use it as a timestamp field

## Checking for Missing Values

In [8]:
print(df_users[df_users['lastLogin'].isnull()])

                          _id  active    createdDate  lastLogin      role  \
97   5ff616a68f142f11dd189163    True  1609963174996        NaN  consumer   
143  5ffe115404929101d0aaebb2    True  1610486100208        NaN  consumer   
148  5ffe115404929101d0aaebb2    True  1610486100208        NaN  consumer   
170  5e27526d0bdb6a138c32b556    True  1579635309795        NaN  consumer   
180  6002475cfb296c121a81b98d    True  1610762076571        NaN  consumer   
..                        ...     ...            ...        ...       ...   
381  60186237c8b50e11d8454d5f    True  1612210743551        NaN  consumer   
382  60186237c8b50e11d8454d5f    True  1612210743551        NaN  consumer   
389  60217fa799409b11fcf899fe    True  1612808103714        NaN  consumer   
420  5fb0a078be5fc9775c1f3945    True  1605410936818        NaN  consumer   
429  5fb0a078be5fc9775c1f3945    True  1605410936818        NaN  consumer   

    signUpSource state  
97         Email    KY  
143        Email    AL  


In [9]:
#62 records have NaN lastLogin values. It could mean that these users have never logged in the past.

In [10]:
df_users['lastLogin'] = df_users['lastLogin'].astype('Int64')

In [11]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1609687530597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI


## Checking for Duplicate Records

In [12]:
duplicateRowsDF = df_users[df_users.duplicated()]
duplicateRowsDF

,_id,active,createdDate,lastLogin,role,signUpSource,state
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,1418998882381,1614963143204,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,1418998882381,1614963143204,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,1418998882381,1614963143204,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,1418998882381,1614963143204,fetch-staff,NaN,NaN


In [13]:
#There are 283 duplicate records

In [14]:
df_users.drop_duplicates()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1609687530597,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,1609687528354,1609687528392,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,1609687479626,1609687479665,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,1609687537564,1609687537599,consumer,Email,WI
...,...,...,...,...,...,...,...
435,5fc961c3b8cfca11a077dd33,True,1607033283936,1614379156799,fetch-staff,Email,NH
455,5fa41775898c7a11a6bcef3e,True,1604589429396,1614873722026,fetch-staff,Email,NaN
456,5fa32b4d898c7a11a6bcebce,True,1604528973309,1614842518047,fetch-staff,Google,AL
462,5964eb07e4b03efd0c0f267b,True,1499785991771,1614884869770,fetch-staff,NaN,IL


## Receipts

In [15]:
df_receipts = pd.read_json("C:/Users/keert/Downloads/json_data/receipts.json/receipts.json", lines=True)

In [16]:
df_receipts.shape

(1119, 15)

In [17]:
df_receipts = df_receipts.explode('rewardsReceiptItemList')

In [18]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6


In [19]:
receipts_itemlist = df_receipts[df_receipts['rewardsReceiptItemList'].notna()]

In [20]:
receipts_itemlist

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,{'$oid': '603cc2bc0a720fde100003e9'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594748000},{'$date': 1614594748000},NaN,{'$date': 1614594749000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [21]:
receipts_itemlist.shape

(6941, 15)

In [22]:
itemlist_df = pd.json_normalize(receipts_itemlist['rewardsReceiptItemList'], max_level=0)

In [23]:
itemlist_df

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [24]:
df_receipts_master = pd.merge(receipts_itemlist, itemlist_df, left_index=True, right_index=True)

In [25]:
#Converting timestamp into integer and cleaning json objects into readable format
df_receipts_master['_id'] = df_receipts_master['_id'].str.get('$oid')
df_receipts_master['createDate'] = df_receipts_master['createDate'].str.get('$date').astype('Int64')
df_receipts_master['finishedDate'] = df_receipts_master['finishedDate'].str.get('$date').astype('Int64')
df_receipts_master['dateScanned'] = df_receipts_master['dateScanned'].str.get('$date').astype('Int64')
df_receipts_master['modifyDate'] = df_receipts_master['modifyDate'].str.get('$date').astype('Int64')
df_receipts_master['pointsAwardedDate'] = df_receipts_master['pointsAwardedDate'].str.get('$date').astype('Int64')
df_receipts_master['purchaseDate'] = df_receipts_master['purchaseDate'].str.get('$date').astype('Int64')


In [26]:
df_receipts_master

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned_x,purchaseDate,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned_y,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1609687531000,1609687536000,1609687531000,500.0,1609632000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1609687483000,1609687488000,1609687483000,150.0,1609601083000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1609687483000,1609687488000,1609687483000,150.0,1609601083000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,<NA>,1609687542000,<NA>,5.0,1609632000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1609687534000,1609687539000,1609687534000,5.0,1609632000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,603cc2bc0a720fde100003e9,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594748000,1614594748000,<NA>,1614594749000,<NA>,25.0,1597622400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,<NA>,1614594148000,<NA>,25.0,1597622400000,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,<NA>,1614594148000,<NA>,25.0,1597622400000,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,<NA>,1614604049000,<NA>,25.0,1597622400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_receipts_master.to_csv('Downloads/receipts_master.csv')

## End